In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

ModuleNotFoundError: No module named 'xgboost'

## DATA

In [3]:
def read_csv_files(folder_path, file_names):
    dataframes = {}
    for file_name in file_names:
        # Construir la ruta completa del archivo
        csv_path = os.path.join(folder_path, file_name)
        
        # Verificar si el archivo existe en la ruta especificada
        if os.path.exists(csv_path):
            # Leer el archivo CSV
            df = pd.read_csv(csv_path)            
            # Añadir el DataFrame al diccionario
            dataframes[file_name] = df
            
            # Mostrar las primeras filas del DataFrame para verificar
            print(f"DataFrame de {file_name}:")
            
        else:
            print(f"El archivo {csv_path} no se encuentra en la ruta especificada.")
    
    return dataframes


folder_path = r"C:\Users\Sofia\Documentos\IMPORTANTES\Estudio\ML-Templates\Time_Series"
file_names = ["PJME_hourly.csv"]
dataframes = read_csv_files(folder_path, file_names)

data = dataframes["PJME_hourly.csv"]

DataFrame de PJME_hourly.csv:


In [4]:
# SETTING DATE VARIABLE
data = data.set_index('Datetime')
data.index = pd.to_datetime(data.index)
data

,PJME_MW
Datetime,
2002-12-31 01:00:00,26498.0
2002-12-31 02:00:00,25147.0
2002-12-31 03:00:00,24574.0
2002-12-31 04:00:00,24393.0
2002-12-31 05:00:00,24860.0
...,...
2018-01-01 20:00:00,44284.0
2018-01-01 21:00:00,43751.0
2018-01-01 22:00:00,42402.0


## Data Exploratory Analysist

In [ ]:
data.plot(style='.',
        figsize=(15, 5),
        color=color_pal[0],
        title='PJME Energy Use in MW')
plt.show()